### Vanilla CTR Zeroshot

In [1]:
import pickle
import json
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv
import re

In [2]:
inference_path = 'zeroshot_only_description_inference_valid.json'
with open(inference_path, 'r') as f:
    inference_dict = json.load(f)
print(len(inference_dict))

928


In [3]:
ratings_path = '../fashion_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

,item,user,rating,timestamp
0,0,0,4.0,1452816000
1,1,0,4.0,1453766400
2,2,0,3.0,1453766400
3,3,0,4.0,1453766400
4,4,0,4.0,1453766400


In [4]:
pattern = r"Prediction:(?:\s+)?(Yes|No)\b"
invalid_users = []
cnt = cntInvalid = cntY = cntN = 0
user_inference_dict = dict()
for user, inference in inference_dict.items():
    cnt += 1
    # print(user, inference)
    # Use re.findall to find all matches in the string
    matches = re.findall(pattern, inference, re.IGNORECASE)
    if len(matches) == 0:
        cntInvalid += 1
        invalid_users.append(user)
        user_inference_dict[user] = 0
    for match in matches:
        if match == 'Yes':
            cntY += 1
            user_inference_dict[user] = 1
            break
        elif match == 'No':
            cntN += 1
            user_inference_dict[user] = 0
            break
    # print('*'*100)
    # if cnt == 10:
    #     break
print(f"Yes: {cntY} -- No: {cntN} -- Invalid: {cntInvalid}")
print("invalid_users:", invalid_users)
print(len(user_inference_dict))

Yes: 541 -- No: 253 -- Invalid: 134
invalid_users: ['21', '34', '36', '39', '40', '55', '60', '61', '66', '68', '81', '91', '115', '117', '138', '155', '157', '179', '181', '189', '206', '221', '240', '246', '250', '255', '262', '269', '271', '286', '306', '316', '321', '340', '357', '382', '386', '389', '432', '434', '440', '451', '455', '458', '460', '468', '476', '481', '482', '483', '486', '487', '488', '490', '493', '502', '521', '546', '563', '571', '596', '601', '606', '638', '642', '660', '661', '672', '687', '714', '723', '725', '734', '740', '751', '756', '764', '773', '785', '786', '788', '816', '829', '830', '849', '855', '861', '873', '883', '887', '893', '900', '911', '920', '924', '925', '927', '929', '945', '960', '973', '977', '991', '994', '996', '1024', '1029', '1034', '1043', '1044', '1058', '1062', '1068', '1099', '1100', '1121', '1127', '1129', '1144', '1150', '1153', '1154', '1162', '1166', '1184', '1215', '1229', '1230', '1231', '1236', '1237', '1238', '1241', '

In [13]:
user_test_movie_rating_df = ratings_df.groupby('user').nth(-1)
user_test_movie_rating_df.head()

,item,user,rating,timestamp
7,7,0,3.0,1500854400
17,17,1,5.0,1470096000
25,25,2,4.0,1462924800
33,25,3,4.0,1495065600
41,25,4,5.0,1509580800


In [14]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_test_movie_rating_df[user_test_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[203  87]
 [266 692]]
AUC: 0.7111691022964509
Precision: 0.8883183568677792
Recall: 0.7223382045929019
Accuracy: 0.7171474358974359


In [5]:
user_valid_movie_rating_df = ratings_df.groupby('user').nth(-2)
user_valid_movie_rating_df.head()

,item,user,rating,timestamp
6,6,0,2.0,1453766400
16,16,1,5.0,1468195200
24,24,2,4.0,1462924800
32,24,3,4.0,1495065600
40,24,4,5.0,1509580800


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
y_true = []
y_pred = []

fp_users = []
fn_users = []

for user, inference in user_inference_dict.items():
    original_rating = user_valid_movie_rating_df[user_valid_movie_rating_df['user'] == int(user)]['rating'].values[0]
    # print(original_rating)
    if original_rating > 3:
        y_true.append(1)
        if inference == 0:
            fn_users.append(user)
    else:
        y_true.append(0)
        if inference == 1:
            fp_users.append(user)
    y_pred.append(inference)

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

auc_roc = roc_auc_score(y_true, y_pred)
print("AUC:", auc_roc)

precision = precision_score(y_true, y_pred)
print("Precision:", precision)

recall = recall_score(y_true, y_pred)
print("Recall:", recall)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[149  93]
 [238 448]]
AUC: 0.6343818519143194
Precision: 0.8280961182994455
Recall: 0.6530612244897959
Accuracy: 0.6433189655172413
